<a href="https://colab.research.google.com/github/Ngoson2004/Vector-search_Engine/blob/main/Vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. First steps

In [ ]:
!pip install datasets
!pip install torch
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

ds = load_dataset("dmis-lab/MedLFQA", streaming=True, split='test')
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/568 [00:00<?, ?B/s]

IterableDataset({
    features: ['Question', 'Free_form_answer', 'Must_have', 'Nice_to_have'],
    num_shards: 5
})

In [ ]:
import pandas as pd

df = pd.DataFrame(ds)
df.head()

,Question,Free_form_answer,Must_have,Nice_to_have
0,Are benign brain tumors serious?,"A benign brain tumor, while not cancerous, can...",[Benign brain tumors are not cancerous and do ...,[Benign brain tumors grow slowly and often hav...
1,Are boils and carbuncles curable?,"Yes, boils and carbuncles are curable. Boils a...",[Boils and carbuncles are skin infections that...,[Recurring boils or carbuncles may indicate an...
2,Are bone cysts serious?,Bone cysts are not typically life-threatening ...,[Bone cysts are typically benign and are not u...,[Bone cysts are most prevalent in children and...
3,Are cold sores a herpes virus?,"Yes, cold sores are caused by a type of the he...",[Cold sores are caused by the herpes simplex v...,"[Herpes simplex virus type 2 (HSV-2), typicall..."
4,Are dental abscesses serious?,"Yes, dental abscesses can be serious if not pr...","[A dental abscess can lead to serious, life-th...","[Regular dental hygiene practice, like brushin..."


# 2. Model loading

## GPT-4o

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
model = OpenAI()

# 3. Database connection

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.operations import SearchIndexModel
import os

uri = os.environ.get('MONGO_URI')
client = MongoClient(uri)
print("Connected successfully.")

db = client.get_database('COS20015')
collection = db.get_collection('OpenAILFQA')

Connected successfully.


In [ ]:
first_embedding_field_name = "question_embedd"
vector_search_index_name_text = "openai_vector_index"

In [ ]:
def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    # Call OpenAI API to get the embedding
    embedding = model.embeddings.create(
        input=text,
        model="text-embedding-3-small").data[0].embedding
    return embedding

# 4. Database creation

## GPT-4o

In [ ]:
df['question_embedd'] = df.apply(lambda row: get_embedding(row["Question"]), axis=1)
df.head()

,Question,Free_form_answer,Must_have,Nice_to_have,question_embedd
0,Are benign brain tumors serious?,"A benign brain tumor, while not cancerous, can...",[Benign brain tumors are not cancerous and do ...,[Benign brain tumors grow slowly and often hav...,"[-0.025564422830939293, 0.035564322024583817, ..."
1,Are boils and carbuncles curable?,"Yes, boils and carbuncles are curable. Boils a...",[Boils and carbuncles are skin infections that...,[Recurring boils or carbuncles may indicate an...,"[-0.013193557970225811, 0.016036227345466614, ..."
2,Are bone cysts serious?,Bone cysts are not typically life-threatening ...,[Bone cysts are typically benign and are not u...,[Bone cysts are most prevalent in children and...,"[0.010376568883657455, 0.040363043546676636, 0..."
3,Are cold sores a herpes virus?,"Yes, cold sores are caused by a type of the he...",[Cold sores are caused by the herpes simplex v...,"[Herpes simplex virus type 2 (HSV-2), typicall...","[-0.05114934220910072, -0.030523264780640602, ..."
4,Are dental abscesses serious?,"Yes, dental abscesses can be serious if not pr...","[A dental abscess can lead to serious, life-th...","[Regular dental hygiene practice, like brushin...","[-0.01886100508272648, 0.03961954265832901, 0...."


In [ ]:
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000000fb'), 'opTime': {'ts': Timestamp(1730188710, 3), 't': 251}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1730188710, 3), 'signature': {'hash': b'\xae\xd3\xc2v_\xa3\xf2V_6\\V\xabu\xca`D\x1br/', 'keyId': 7374128547067068423}}, 'operationTime': Timestamp(1730188710, 3)}, acknowledged=True)

In [ ]:
import numpy as np

for record in df.to_dict(orient='records'):
  # for key, value in record.items():
  #   if not isinstance(value, list):
  #     processed_list = value.tolist()
  #     record[key] = processed_list
  collection.insert_one(record)

print("Data imported successfully")

Data imported successfully


# 5. Vector search index

In [ ]:
vector_search_index_model = SearchIndexModel(
    definition={
        "mappings": { # describes how fields in the database documents are indexed and stored
            "dynamic": True, # automatically index new fields that appear in the document
            "fields": { # properties of the fields that will be indexed.

                    first_embedding_field_name: {
                    "dimensions": 1536, # size of the vector.
                    "similarity": "cosine", # algorithm used to compute the similarity between vectors
                    "type": "knnVector",
                    }
            },
        }
    },
    name=vector_search_index_name_text, # identifier for the vector search index
)

In [ ]:
# Check if the index already exists
index_exists = False
for index in collection.list_indexes():
    print(index)
    if index['name'] == vector_search_index_name_text:
        index_exists = True
        break

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [ ]:
import time

if not index_exists:
    try:
        result = collection.create_search_index(model=vector_search_index_model)
        print("Creating index...")
        time.sleep(20)  # Sleep for 20 seconds, adding sleep to ensure vector index has compeleted inital sync before utilization
        print("Index created successfully:", result)
    except Exception as e:
        print(f"Error creating vector search index: {str(e)}")
else:
    print(f"Index '{vector_search_index_name_text}' already exists.")


Creating index...
Index created successfully: openai_vector_index


# 6. Compose vector search query

In [ ]:
def vector_search(user_query, db, collection, vector_index=vector_search_index_name_text):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    db (MongoClient.database): The database object.
    collection (MongoCollection): The MongoDB collection to search.
    additional_stages (list): Additional aggregation stages to include in the pipeline.

    Returns:
    list: A list of matching documents.
    """
    # tokeniser.pad_token = tokeniser.eos_token
    # first_query_embedding = tokeniser.encode(user_query, padding='max_length', truncation=True, max_length=50)
    # second_query_embedding = tokeniser.encode(user_query, padding='max_length', truncation=True, max_length=976)

    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search stage
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index, # specifies the index to use for the search
            "queryVector": query_embedding, # the vector representing the query
            "path": first_embedding_field_name, # field in the documents containing the vectors to search against
            "numCandidates": 150, # number of candidate matches to consider
            "limit": 10 # return top 20 matches
        }
    }

    # Define the aggregate pipeline with the vector search stage and additional stages
    # pipe = [vector_search_stage]

    # Execute the search
    result1 = collection.aggregate([vector_search_stage])

    return list(result1)


In [ ]:
from IPython.display import display, HTML

def handle_user_query(query, db, collection, max_length=576, max_new_tokens=512):
    # Assuming vector_search returns a list of dictionaries with keys 'title' and 'plot'
    knowledge = vector_search(query, db, collection)

    # Check if there are any results
    if not knowledge:
        return "No results found.", "No source information available."

    context_str = '\n'.join(f"{j}. {k['Free_form_answer']}" for j,k in enumerate(knowledge))
    # QA_input = {
    #     'question': query,
    #     'context': context_str
    # }
    # system_response = model(QA_input)

    # system_response = model(f"Answer this query: {query}\nUse the following numbered list of contexts:\n{context_str}", max_new_tokens=max_new_tokens)
    # system_response = model.generate_content(f"Answer this query: {query}\nUse the following numbered list of contexts:\n{context_str}")
    completion = model.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert in bio-medical"},
        {
            "role": "user",
            "content": f"Answer this user query: {query} with the following context:\n{context_str}"
        }
      ]
    )
    system_response = completion.choices[0].message.content

    # Print User Question, System Response, and Source Information
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")
    print(f"- Source Information:\n{context_str}\n")


    # Return structured response and source info as a string

In [ ]:
query = """
What is malaria?
"""

print(len(vector_search(query, db, collection)))
handle_user_query(query, db, collection)

10
- User Question:

What is malaria?


- System Response:
Malaria is a serious and potentially life-threatening disease primarily caused by the Plasmodium parasite, which is transmitted through the bite of infected Anopheles mosquitoes. There are five species of Plasmodium that can infect humans: Plasmodium falciparum, Plasmodium vivax, Plasmodium ovale, Plasmodium malariae, and Plasmodium knowlesi, with Plasmodium falciparum being the most deadly. Once inside the human body, these parasites multiply in the liver and then infect red blood cells. 

Symptoms of malaria typically appear 10–15 days after being bitten and can include fever, chills, and flu-like illness. If untreated, malaria can rapidly become severe, affecting vital organs such as the kidneys and brain, and can lead to serious health complications or death. However, with prompt diagnosis and treatment using anti-malarial medications, most individuals can fully recover from the disease.

- Source Information:
0. Malaria is